In [1]:
import caffe
import cv2
from skimage import transform as trans
import numpy as np

imgSize = [112, 96]
# the landmarks for template image, used for alignment
coord5points = [30.2946, 65.5318, 48.0252, 33.5493, 62.7299,
                51.6963, 51.5014, 71.7366, 92.3655, 92.2041]
coord5points = np.reshape(coord5points, (2,5))
coord5points = coord5points.transpose()

img = cv2.imread('test.jpg')
# the landmarks of test image
facial5points = [114.107, 160.884, 139.439, 118.294, 158.177,
                 110.901, 110.388, 132.868, 157.38 , 156.312]
facial5points = np.reshape(facial5points, (2,5))
facial5points = facial5points.transpose()

# use similar transform to align the input image
tform = trans.SimilarityTransform()                                                                                                                                                  
tform.estimate(facial5points, coord5points)
M = tform.params[0:2,:]
warped = cv2.warpAffine(img, M, (96,112), borderValue = 0.0)

# cv2.imshow('img', warped)
# cv2.waitKey(-1)

# pre-processing the aligned image
img = (warped-127.5)/128
img = img[:,:,(2,1,0)]
img = img.transpose((2, 0, 1)) # re-order dimensions
img = np.expand_dims(img, axis=0) # add dimension for batch

print(img.shape)


(1, 3, 112, 96)


In [2]:
MODEL_DEF = '/path/to/face_deploy.prototxt'
MODEL_WEIGHTS = '/path/to/face_model.caffemodel'
net = caffe.Net(MODEL_DEF, MODEL_WEIGHTS, caffe.TEST)

In [3]:
# output = net.predict(img)
net.blobs['data'].reshape(1, 3, 112, 96)
net.blobs['data'].data[...] = img
output = net.forward()
print output

{'fc5': array([[  1.24699563e-01,   1.49494159e+00,  -2.92533612e+00,
          7.59052098e-01,  -7.30646253e-01,   1.61084592e+00,
         -6.68065429e-01,   6.35521352e-01,  -1.60791314e+00,
          7.49957860e-01,  -1.07159889e+00,  -2.48870945e+00,
          1.97457719e+00,   2.35225409e-02,   8.65168631e-01,
         -2.83539206e-01,  -2.39356327e+00,  -2.21189812e-01,
          1.00929618e+00,  -3.41389298e-01,   3.92379165e-01,
          1.28640401e+00,  -1.91298044e+00,   4.05675322e-01,
          1.53568709e+00,   1.28611076e+00,   8.99530888e-01,
         -1.94289279e+00,   7.19263673e-01,   1.52965927e+00,
          1.29637823e-01,  -1.03224561e-01,   6.34733081e-01,
          3.61981153e-01,   7.62480855e-01,   4.68872964e-01,
         -8.70131850e-01,  -2.64388323e-01,  -6.63900971e-01,
         -4.46309149e-01,  -6.91666484e-01,  -2.31525350e+00,
         -1.67957574e-01,   1.00187004e+00,  -3.74299586e-01,
         -8.00313801e-02,   9.35247898e-01,   3.98338258e-01,


In [4]:


from __future__ import division, print_function
from keras import backend as K
from keras.layers import Input, Add
from keras.layers.core import Activation, Dense, Flatten
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Convolution2D, ZeroPadding2D
from keras.layers import Input, Conv2D, Activation, GlobalAvgPool2D, Dense, BatchNormalization, PReLU, Merge, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
import numpy as np
import os
import re
from keras.optimizers import SGD


# be careful, you should use theano as backend, and set data_format='channels_last' in ~/.keras/keras.json
def keras_centerloss(input_data):
    # conv1a, conv1b
    l = Conv2D(32, (3, 3),  name='conv1a', padding='valid')(input_data)
    l = PReLU(name='relu1a', shared_axes=[2,3])(l)

    l = Conv2D(64, (3, 3),  name='conv1b', padding='valid')(l)
    l = PReLU(name='relu1b', shared_axes=[2,3])(l)
    pool1 = MaxPooling2D(pool_size=(2, 2), strides=2, data_format=None)(l)

    # conv2_1, conv2_2
    l = Conv2D(64, (3, 3), padding='same', name='conv2_1')(pool1)
    l = PReLU(name='relu2_1', shared_axes=[2,3])(l)
    l = Conv2D(64, (3, 3), padding='same', name='conv2_2')(l)
    l = PReLU(name='relu2_2', shared_axes=[2,3])(l)
    res2_2 = Add()([pool1, l])

    # conv2
    l = Conv2D(128, (3,3), name='conv2', padding='valid')(res2_2)
    l = PReLU(name='relu2', shared_axes=[2,3])(l)
    pool2 = MaxPooling2D(pool_size=(2,2), strides=2, data_format=None)(l)


    # conv3_1, conv3_2
    l = Conv2D(128, (3, 3), padding='same', name='conv3_1')(pool2)
    l = PReLU(name='relu3_1', shared_axes=[2,3])(l)
    l = Conv2D(128, (3, 3), padding='same', name='conv3_2')(l)
    l = PReLU(name='relu3_2', shared_axes=[2,3])(l)
    res3_2 = Add()([pool2, l])

    # conv3_3, conv3_4
    l = Conv2D(128, (3,3), padding='same', name='conv3_3')(res3_2)
    l = PReLU(name='relu3_3', shared_axes=[2,3])(l)
    l = Conv2D(128, (3,3), padding='same', name='conv3_4')(l)
    l = PReLU(name='relu3_4', shared_axes=[2,3])(l)
    res3_4 = Add()([res3_2, l])

    # conv3
    l = Conv2D(256, (3,3), name='conv3')(res3_4)
    l = PReLU(name='relu3', shared_axes=[2,3])(l)
    pool3 = MaxPooling2D(pool_size=(2,2), strides=2, data_format=None)(l)

    # conv4_1, conv4_2
    l = Conv2D(256, (3,3), padding='same', name='conv4_1')(pool3)
    l = PReLU(name='relu4_1', shared_axes=[2,3])(l)
    l = Conv2D(256, (3,3), padding='same', name='conv4_2')(l)
    l = PReLU(name='relu4_2', shared_axes=[2,3])(l)
    res4_2 = Add()([pool3, l])

    # conv4_3, conv4_4
    l = Conv2D(256, (3,3), padding='same', name='conv4_3')(res4_2)
    l = PReLU(name='relu4_3', shared_axes=[2,3])(l)
    l = Conv2D(256, (3,3), padding='same', name='conv4_4')(l)
    l = PReLU( name='relu4_4',  shared_axes=[2,3])(l)
    res4_4 = Add()([res4_2, l])

    # conv4_5, conv_4_6
    l = Conv2D(256, (3,3), padding='same', name='conv4_5')(res4_4)
    l = PReLU( name='relu4_5', shared_axes=[2,3])(l)
    l = Conv2D(256, (3,3), padding='same', name='conv4_6')(l)
    l = PReLU( name='relu4_6',  shared_axes=[2,3])(l)
    res4_6 = Add()([res4_4, l])

    # conv4_7, conv_4_8
    l = Conv2D(256, (3,3), padding='same', name='conv4_7')(res4_6)
    l = PReLU( name='relu4_7',  shared_axes=[2,3])(l)
    l = Conv2D(256, (3,3), padding='same', name='conv4_8')(l)
    l = PReLU( name='relu4_8',  shared_axes=[2,3])(l)
    res4_8 = Add()([res4_6, l])

    # conv4_9, conv_4_10
    l = Conv2D(256, (3,3), padding='same', name='conv4_9')(res4_8)
    l = PReLU( name='relu4_9',  shared_axes=[2,3])(l)
    l = Conv2D(256, (3,3), padding='same', name='conv4_10')(l)
    l = PReLU( name='relu4_10',  shared_axes=[2,3])(l)
    res4_10 = Add()([res4_8, l])

    # conv4
    l = Conv2D(512, (3,3), name='conv4')(res4_10)
    l = PReLU( name='relu4',  shared_axes=[2,3])(l)
    pool4 = MaxPooling2D(pool_size=(2,2), strides=2, data_format=None)(l)

    # conv5_1 conv5_2
    l = Conv2D(512, (3,3), padding='same', name='conv5_1')(pool4)
    l = PReLU( name='relu5_1', shared_axes=[2,3])(l)
    l = Conv2D(512, (3,3), padding='same', name='conv5_2')(l)
    l = PReLU( name='relu5_2', shared_axes=[2,3])(l)
    res5_2 = Add()([pool4, l])

    # conv5_3 conv5_4
    l = Conv2D(512, (3,3), padding='same', name='conv5_3')(res5_2)
    l = PReLU( name='relu5_3', shared_axes=[2,3])(l)
    l = Conv2D(512, (3,3), padding='same', name='conv5_4')(l)
    l = PReLU( name='relu5_4', shared_axes=[2,3])(l)
    res5_4 = Add()([res5_2, l])

    # conv5_5 conv5_6
    l = Conv2D(512, (3,3), padding='same', name='conv5_5')(res5_4)
    l = PReLU( name='relu5_5', shared_axes=[2,3])(l)
    l = Conv2D(512, (3,3), padding='same', name='conv5_6')(l)
    l = PReLU( name='relu5_6', shared_axes=[2,3])(l)
    res5_6 = Add()([res5_4, l])


    # full connect
    fc5 = Flatten()(res5_6)
    fc5 = Dense(512, name='fc5')(fc5)

    return fc5



Using Theano backend.


In [5]:
data = Input(shape=(3, 112, 96))
output = keras_centerloss(data)
centerloss = Model(input=data, output=output)

/home/cupwater/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Elemwise{a..., inputs=/input_1)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
for layer_name in net.params.keys():    
    if 'fc' in layer_name or 'conv' in layer_name or 'relu' in layer_name:
        model_layer = centerloss.get_layer(name=layer_name)
#         print(layer_name + ': ' + str(model_layer.output_shape))
        if 'fc' in layer_name:
            weights_w = np.copy(net.params[layer_name][0].data)
            weights_w = np.transpose(weights_w)
            weights_b = np.copy(net.params[layer_name][1].data)
            model_layer.set_weights([weights_w, weights_b])
        if 'conv' in layer_name:
            weights = np.copy(net.params[layer_name][0].data)
#             print(weights.shape)
            biases = np.copy(net.params[layer_name][1].data)
            for i in range(weights.shape[0]): # go through each filter
                for j in range(weights.shape[1]): # go through each channel
                    weights[i, j] = np.rot90(weights[i, j], 2) # rotate it (twice)
            weights = np.transpose(weights, (2,3,1,0)) 
            model_layer.set_weights([weights, biases])
        if 'relu' in layer_name:
            weights = np.copy(net.params[layer_name][0].data)
            weights = np.reshape(weights, (weights.shape[0],1,1))
            model_layer.set_weights([weights])
            

    else:
        continue

In [7]:
feas = centerloss.predict(img)
print(feas)

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


[[  1.24699742e-01   1.49494123e+00  -2.92533636e+00   7.59052992e-01
   -7.30645776e-01   1.61084545e+00  -6.68065727e-01   6.35521054e-01
   -1.60791278e+00   7.49958217e-01  -1.07159936e+00  -2.48870921e+00
    1.97457719e+00   2.35222280e-02   8.65168631e-01  -2.83539712e-01
   -2.39356232e+00  -2.21189827e-01   1.00929594e+00  -3.41389120e-01
    3.92379344e-01   1.28640354e+00  -1.91298163e+00   4.05675501e-01
    1.53568649e+00   1.28611100e+00   8.99531305e-01  -1.94289315e+00
    7.19264030e-01   1.52965939e+00   1.29637003e-01  -1.03224114e-01
    6.34732664e-01   3.61981690e-01   7.62480915e-01   4.68874395e-01
   -8.70131612e-01  -2.64388680e-01  -6.63901210e-01  -4.46309149e-01
   -6.91666007e-01  -2.31525326e+00  -1.67957753e-01   1.00186956e+00
   -3.74299705e-01  -8.00315291e-02   9.35248494e-01   3.98337662e-01
    1.28901929e-01  -1.95262837e+00   8.90405655e-01  -2.27261752e-01
   -9.12093461e-01   2.52208769e-01  -4.19162840e-01   1.96155041e-01
    3.57572198e-01  

In [10]:
centerloss.save_weights('kerasCenterloss.h5')